## Run these first three cells to get external code needed by the spreadsheet and import Python modules

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO-modules.git

In [ ]:
%run ECHO-modules/DataSet.py

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

from IPython.display import display


## Run this cell.  It reads the CSV file of prisons and provides a dropdown list to choose from

In [ ]:
# Import FRS IDs from california prisons 
df = pd.read_csv('reformatted_ca_validated.csv')

prison_dict = {}
for idx, row in df.iterrows():
    name = row['NAME']
    frs_id = row['Total_FRS_IDS']
    if ( name in prison_dict ):
        prison_dict[ name ].append( frs_id )
    else:
        prison_dict[ name ] = [ frs_id ]

prison_widget = widgets.Select(
    options=list( prison_dict.keys() ),
    description='Prison:',
    layout=Layout(width='70%', height='80px'),
    disabled=False
)
display( prison_widget )

## This cell will query the Stonybrook database for ECHO_EXPORTER facility information for the selected prison

In [ ]:
prison_name = prison_widget.value
prison_ids = prison_dict[ prison_name ]

echo_ds = DataSet( name='ECHO Exporter', idx_field='REGISTRY_ID', 
                table_name='ECHO_EXPORTER', echo_type=None,
                date_field=None, date_format=None)
echo_data = echo_ds.get_data( ee_ids=prison_ids )
echo_data

## Show the facilities on a map

In [ ]:
# Let's show a quick map of your area and the facilities in it
# To-do:  Add some more ECHO_EXPORTER information in the markers.

# Put some information with the marker to show the programs that track the facility.
def marker_text( row ):
    text = ""
    if ( type( row['FAC_NAME'] == str )) :
        try:
            text = row["FAC_NAME"] + ' - '
        except TypeError:
            print( "A facility was found without name. ")
        if ( row['AIR_FLAG'] == 'Y' ):
            text += 'CAA, ' 
        if ( row['NPDES_FLAG'] == 'Y' ):
            text += 'CWA, ' 
        if ( row['SDWIS_FLAG'] == 'Y' ):
            text += 'SDWIS, ' 
        if ( row['RCRA_FLAG'] == 'Y' ):
            text += 'RCRA, ' 
        if ( row['TRI_FLAG'] == 'Y' ):
            text += 'TRI, ' 
        if ( row['GHG_FLAG'] == 'Y' ):
            text += 'GHG, ' 
    return text
    
def mapper(df):
    # Initialize the map
    center = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]]
    m = folium.Map(
        location = center,
    )

    # Add a clickable marker for each facility
    i = 0
    for index, row in df.iterrows():
        # Make sure the FAC_NAME is not NaN, which is interpreted as a number.
        if ( type( row['FAC_NAME'] == str )) :
            folium.Marker(
                location = [row["FAC_LAT"], row["FAC_LONG"]],
                popup = marker_text( row )).add_to(m)
            i += 1
        if ( i > 400 ):    # The map won't display with too many markers.
            break
            
    bounds = m.get_bounds()
    m.fit_bounds(bounds)
    
    # Show the map
    return m

df_to_map = echo_data
if ( len(df_to_map) > 400 ):
    df_to_map = echo_data[echo_data["FAC_QTRS_WITH_NC"] > 0].sort_values(by="FAC_QTRS_WITH_NC", ascending=False)
    
map_of_facilities = mapper(df_to_map)
map_of_facilities

## Run this cell to get a dropdown list containing the programs that are tracked for these facilities

In [ ]:
# Create a DataSet object for each of the programs we track.  
# Initialize each one with the information it needs to do its query
# of the database.
# Store the DataSet objects in a dictionary with keys being the
# friendly names of the program, which will be used in selection
# widgets.

data_sets = {}
ds = DataSet( name='RCRA Violations', idx_field='ID_NUMBER', 
                table_name='RCRA_VIOLATIONS', echo_type="RCRA",
                date_field='DATE_VIOLATION_DETERMINED', date_format='%m/%d/%Y')
data_sets[ ds.name ] = ds
ds = DataSet( name='RCRA Inspections', idx_field='ID_NUMBER', 
                table_name='RCRA_EVALUATIONS', echo_type="RCRA",
                date_field='EVALUATION_START_DATE', date_format='%m/%d/%Y')
data_sets[ ds.name ] = ds
ds = DataSet( name='RCRA Enforcements',  echo_type="RCRA",
                table_name='RCRA_ENFORCEMENTS', idx_field='ID_NUMBER', 
                date_field='EVALUATION_START_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
# Special sql queries needed for Air Inspections and Air Enforcements
my_sql = "select * from `ICIS_FEC_EPA_INSPECTIONS` where REGISTRY_ID in "
ds = DataSet( name='Air Inspections', echo_type="AIR",
                table_name='ICIS_FEC_EPA_INSPECTIONS', idx_field='REGISTRY_ID', 
                date_field='ACTUAL_END_DATE', date_format='%m/%d/%Y', sql=my_sql )
data_sets[ ds.name ] = ds
my_sql = "select * from `CASE_FACILITIES` CF, `CASE_ENFORCEMENTS` CE " + \
            " where CE.HQ_DIVISION = 'AIR' and CE.CASE_NUMBER = CF.CASE_NUMBER and " + \
            " CF.REGISTRY_ID in "
ds = DataSet( name='Air Enforcements',  echo_type="AIR",
                table_name='CASE_ENFORCEMENTS', idx_field='REGISTRY_ID',
                date_field='FISCAL_YEAR', date_format='%Y', sql=my_sql )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Violations',  echo_type="AIR",
                table_name='ICIS-AIR_VIOLATION_HISTORY', idx_field='PGM_SYS_ID', 
                date_field='HPV_DAYZERO_DATE', date_format='%m-%d-%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Formal Actions', echo_type="AIR",
                table_name='ICIS-AIR_FORMAL_ACTIONS', idx_field='PGM_SYS_ID',
                date_field='SETTLEMENT_ENTERED_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Compliance', echo_type="AIR",
                table_name='ICIS-AIR_FCES_PCES', idx_field='PGM_SYS_ID',
                date_field='ACTUAL_END_DATE', date_format='%m-%d-%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Water Quarterly Violations', echo_type="NPDES",
                table_name='NPDES_QNCR_HISTORY', idx_field='NPDES_ID',
                date_field='YEARQTR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Clean Water Inspections', echo_type="NPDES",
                table_name='NPDES_INSPECTIONS', idx_field='NPDES_ID',
                date_field='ACTUAL_END_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Clean Water Enforcements', echo_type="NPDES",
                table_name='NPDES_FORMAL_ENFORCEMENT_ACTIONS', idx_field='NPDES_ID',
                date_field='SETTLEMENT_ENTERED_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Site Visits', echo_type="SDWA",
                table_name='SDWA_SITE_VISITS', idx_field='PWSID',
                date_field='SITE_VISIT_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Enforcements', echo_type="SDWA",
                table_name='SDWA_ENFORCEMENTS', idx_field='PWSID',
                date_field='ENFORCEMENT_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Public Water Systems', echo_type="SDWA",
                table_name='SDWA_PUB_WATER_SYSTEMS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Violations', echo_type="SDWA",
                table_name='SDWA_VIOLATIONS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Serious Violators', echo_type="SDWA",
                table_name='SDWA_SERIOUS_VIOLATORS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Return to Compliance', echo_type="SDWA",
                table_name='SDWA_RETURN_TO_COMPLIANCE', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )

# Only list the data set if it has the correct flag set.
data_set_choices = []
for k, v in data_sets.items():
    if ( v.has_echo_flag( echo_data ) ):
        data_set_choices.append( k )

# data_set_choices = list( data_sets.keys() )

data_set_widget=widgets.Dropdown(
    options=list(data_set_choices),
    description='Data sets:',
    disabled=False,
) 
display(data_set_widget)


## Run this cell to get the data for the selected program

In [ ]:
program = data_sets[ data_set_widget.value ]

ids_string = program.echo_type + '_IDS'
ids = list()
for index, value in facility[ ids_string ].items():
    try:
        for this_id in value.split():
            ids.append( this_id )
    except ( KeyError, AttributeError ) as e:
        pass

program_data = None
# We need to provide a custom sql query and argument for these programs.
if ( program.name == "Air Inspections" or program.name == "Air Enforcements" ):
    # The REGISTRY_ID field is the index of the echo_data
    # Build a string
    registry_ids = facility[facility['AIR_FLAG'] == 'Y'].index.to_list()
    program_data = program.get_data( ee_ids=registry_ids )
else:
    program_data = program.get_data( ee_ids=ids )

program_data

## This section saves the facility data to your computer.
You can return to the dropdown to export additional data files.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
filename = prison_widget.value
fullpath = filename + '-' + program.name + '.csv'
program_data.to_csv( fullpath ) 

print( "Wrote "+filename+" to the Google Colab 'Files' menu as %s" %(fullpath))